In [1]:
from model_functions import analyze_token_sequence, predict, write_midi, get_token_flags
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

PATH_VOCAB = "../0_data/5_vocabs"
PATH_MODELS = "../0_data/7_models"
PATH_MODELS_CONFIG = "../0_data/7_models/config"
PATH_PRED = "../0_data/8_predictions"
PATH_TOKENS = "../0_data/8_predictions/tokens"
PATH_MIDI = "../0_data/8_predictions/midi"

for path in [PATH_PRED, PATH_TOKENS, PATH_MIDI]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "a2" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": True,
    },
    "a3" : {
        "pitch_range": 128,
        "duration_steps": 32,
        "triole_tokens": False,
    },
    "b" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "c" : {
        "pitch_range": 36,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "d" : {
        "pitch_range": 36,
        "duration_steps": 32,
        "triole_tokens": True,
    }
}

In [3]:
model_df = pd.read_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx", index_col="Unnamed: 0")
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1,1024,256,4,6,0,0.001,50,4,yes,717.6662,11.96,1.373842,42,423.106567,0.26,322.08,1
1,a2,1024,256,4,6,0,0.001,50,4,yes,716.0189,11.93,1.140858,49,472.513916,0.28,280.23,1
2,a3,1024,256,4,6,0,0.001,50,4,yes,696.7114,11.61,1.342281,34,252.672455,0.14,320.89,1
3,b,1024,256,4,6,0,0.001,50,4,yes,636.9317,10.62,1.325169,27,306.182312,0.43,322.02,1
4,c,1024,256,4,6,0,0.001,50,4,yes,656.0490,10.93,1.362022,32,810.230347,0.34,321.72,1
5,d,1024,256,4,6,0,0.001,50,4,yes,598.8393,9.98,1.167735,50,486.180145,0.39,282.52,1


In [4]:
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32
    
for index, row in model_df.iterrows():

    # only models that ran
    if row["ran"] == "no" or pd.notnull(row["incorrect_notes"]) or row["runtime"] == "too big - cuda error":
        continue
    
    model_name = row["name"]
    if not os.path.exists(f"{PATH_MIDI}/{model_name}"):
        os.makedirs(f"{PATH_MIDI}/{model_name}")

    # get token flags and duration bins
    token_flags = get_token_flags(vocab_configs[model_name])
    duration_steps = vocab_configs[model_name]["duration_steps"]
    duration_bins = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*duration_steps)+1, TICKS_PER_MIN_DURATION, dtype=int)

    # create tokenizer
    tokenizer = GPT2Tokenizer(
        vocab_file=f"{PATH_VOCAB}/vocab_{model_name}.json", 
        merges_file=f"{PATH_VOCAB}/merges.txt")
    tokenizer.add_special_tokens({'pad_token': 'PAD', 'bos_token': 'BOS', 'eos_token': 'EOS',})
    
    # get vocabulary
    with open(f"{PATH_VOCAB}/vocab_{model_name}.json", "r") as fp:
        vocab = json.load(fp)
    token2word = {token: word for word, token in vocab.items()}

    # load model
    model = GPT2LMHeadModel.from_pretrained(f"{PATH_MODELS_CONFIG}/{model_name}/end_version")
    
    # make predictions save
    output = predict(model, tokenizer, samples=100, max_length=row["max_length"])
    data_generated = {"data": output}
    
    with open(f"{PATH_TOKENS}/{model_name}.json", "w") as fp:
        json.dump(data_generated, fp)
    
    # analyze tokens and save as midi_files
    correct_notes = 0
    incorrect_notes = 0
    for idx, pred in enumerate(output):
        an = analyze_token_sequence(pred, token_flags)
        correct_notes += an["start-pos-pitch-duration"] + an["start-pos-pitch-duration-dtriole"] + an["start-pos-ptriole-pitch-duration"] + an["start-pos-ptriole-pitch-duration-dtriole"]
        incorrect_notes += write_midi(output[idx], token2word, duration_bins, f"{PATH_MIDI}/{model_name}/generated_midi_{idx}.midi")
    
    model_df.at[index,"correct_notes"] = (correct_notes/100).__round__(2)
    model_df.at[index,"incorrect_notes"] = (incorrect_notes/100).__round__(2)
    model_df.at[index,"correct_rate"] = (correct_notes/(correct_notes+incorrect_notes)).__round__(2)
    
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1,1024,256,4,6,0,0.001,50,4,yes,717.6662,11.96,1.373842,42,423.106567,0.26,322.08,1
1,a2,1024,256,4,6,0,0.001,50,4,yes,716.0189,11.93,1.140858,49,472.513916,0.28,280.23,1
2,a3,1024,256,4,6,0,0.001,50,4,yes,696.7114,11.61,1.342281,34,252.672455,0.14,320.89,1
3,b,1024,256,4,6,0,0.001,50,4,yes,636.9317,10.62,1.325169,27,306.182312,0.43,322.02,1
4,c,1024,256,4,6,0,0.001,50,4,yes,656.0490,10.93,1.362022,32,810.230347,0.34,321.72,1
5,d,1024,256,4,6,0,0.001,50,4,yes,598.8393,9.98,1.167735,50,486.180145,0.39,282.52,1


In [5]:
model_df.to_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx")

run "tar chvfz predictions_midi.tar.gz *" in terminal midi folder to create and download zip